In [1]:
import pandas as pd     #importo librería pandas que vamos a usar para el data frame
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
#leo el dataframe
df_tags_for_ML=pd.read_csv('Datos_procesados/df_tags_for_ML.csv')
#elimino duplicados
df_tags_for_ML.drop_duplicates('item_id', inplace= True)
#armo el dataframe que voy a usar para exportar la información obtenida en el algoritmo de ML
df_recommendedgames=df_tags_for_ML[['item_id','item_title']]
df_tags_for_ML.drop('item_title', axis=1, inplace= True )
#ubico los Id como índices, necesario para el algoritmo de ML
df_recommendedgames.set_index('item_id', inplace=True)
df_tags_for_ML.set_index('item_id', inplace=True)

In [3]:
#obtengo los indices
indices=df_tags_for_ML.index
#armo el algoritmo de ML
vector_similitud_coseno=cosine_similarity(df_tags_for_ML.values)
#paso los valores a un dataframe para su posterior procesamiento
df_cos_sim=pd.DataFrame(vector_similitud_coseno, index=indices, columns=indices)

In [24]:
#agrego columnas al dataframe, en donde voy a poner los resultados de la recomendación
for i in range(5):
    df_recommendedgames[f'recoitem_id_{i+1}'] = None

#Agrego los resultados de la recomendación
#voy a iterar por cada fila en el dataframe de recommended games
for index, row in df_recommendedgames.iterrows():
    #armo una lista con la columna del df_cos_sim que corresponde a ese juego. La reordeno de mayor a menor y 
    #obtengo los primeros 6 valores
    lista=df_cos_sim.loc[index].sort_values(ascending=False).head(6).index.to_list()
    #elimino de la lista el mismo juego (a veces aparece, a veces no)
    if index in lista: lista.remove(index)
    #pongo en cada columna el valor del juego
    df_recommendedgames.at[index, 'recoitem_id_1'] = lista[0]
    df_recommendedgames.at[index, 'recoitem_id_2'] = lista[1]
    df_recommendedgames.at[index, 'recoitem_id_3'] = lista[2]
    df_recommendedgames.at[index, 'recoitem_id_4'] = lista[3]
    df_recommendedgames.at[index, 'recoitem_id_5'] = lista[4]


In [30]:
#pongo los nombres de los juegos y elimino las columnas
#agrego columnas al dataframe, en donde voy a poner los resultados de la recomendación
for i in range(5):
    df_recommendedgames[f'recommended_game_{i+1}'] = None
#armo un diccionario con las claves y nombres de juegos
dict_games={}
for index, row in df_recommendedgames.iterrows():
    dict_games[index] = row['item_title']
#cambio los valores
df_recommendedgames['recommended_game_1'] = df_recommendedgames['recoitem_id_1'].map(dict_games)
df_recommendedgames['recommended_game_2'] = df_recommendedgames['recoitem_id_2'].map(dict_games)
df_recommendedgames['recommended_game_3'] = df_recommendedgames['recoitem_id_3'].map(dict_games)
df_recommendedgames['recommended_game_4'] = df_recommendedgames['recoitem_id_4'].map(dict_games)
df_recommendedgames['recommended_game_5'] = df_recommendedgames['recoitem_id_5'].map(dict_games)
#elimino las columnas que no me sirven
df_recommendedgames.drop(['recoitem_id_1', 'recoitem_id_2', 'recoitem_id_3', 'recoitem_id_4', 'recoitem_id_5'], axis=1, inplace=True)

C:\Users\feder\AppData\Local\Temp\ipykernel_3432\1997405321.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recommendedgames['recommended_game_1'] = df_recommendedgames['recoitem_id_1'].map(dict_games)


In [39]:
df_recommendedgames.to_csv('Datos_procesados/df_recommended_gamesxgames.csv')